# sc3nb – SuperCollider3 for python / jupyter notebooks
2017-2019 by Thomas Hermann<br>
contributions by: Ferdinand Schlatt, Fabian Kaupmann, Dennis Reinsch, Micha Steffen Vosse

## Introduction

sc3nb is a python module to enable interaction with and control of SuperCollider3 sound programming language, both sclang and scsynth, from python, and specifically from ipython notebooks resp. jupyter notebooks.

The motivation is to facilitate the programming of auditory displays and interactive sonifications by teaming up python (and particularly numpy, scipy, pandas, matplotlib etc.) for data science and SuperCollider3 for interactive real-time sound rendering.

sc3nb so far enables basic control of supercollider both via OSC messages/bundles, and via commands send via PIPE to sclang that runs as a subprocess. Some features are rudimentary and less tested, and need extensions. It is meant to grow into a backend for the sonification package PySon, and can be used both from jupyter and in standard python software development.

sc3nb is hosted at GitHub https://github.com/thomas-hermann/sc3nb

## Getting Started

In [ ]:
# setup, heading
import numpy as np
import time, random, os
import sc3nb as scn

To startup SuperCollider3 (sclang, which in turn boots the scsynths) use

In [ ]:
sc = scn.startup()  # optionally specify arg sclangpath="/path/to/sclang"

* sclang is started as subprocess and its output is collected from the SC class. 
* If sclang executable is not in $PATH, you can specify it by sclangpath="/path/to/sclang-containing-dir/sclang"

You can see what remained unread on startup by 

In [ ]:
sc.cmdv("7*6", discard_output=False)

To de/activate logging on the jupyter console (resp. stdout) use

In [ ]:
# control your console logging as needed
scn.SC.console_logging = True

## sclang command execution

To send sc3 commands (i.e. program strings) to the language, either use the functions
* **cmd()**: non verbose
* **cmdv()**: verbose, i.e. returning string is collected and output to notebook
* **cmdg()**: send program and get and parse the output

or use the corresponding 
* Jupyter line magics **%sc, %scv, %scg**
* Jupyter cell magics **%%sc, %%scv, %%scg**
which wrap the above functions. See examples below

In [ ]:
# sc.cmd(cmdstr, pyvars)
sc.cmd('"hello".postln')  # check jupyter console for output

This can be more conveniently written as

In [ ]:
%sc "sc3nb".postln  // output on console

Cell magics can be placed within code as shown here:

In [ ]:
%sc x = Synth.new(\default, [\freq, 100])
for p in range(1, 10):  # a bouncing ball
    time.sleep(1/p)
    %sc Synth.new(\s1, [\freq, 200])  // this is sc cell so use sc3 comments instead of #
%sc x.free

use cmdv If sclang output should be displayed as cell output

In [ ]:
sc.cmdv('Synth.new(\default)')  # a tone should play

* to stop all playing synths either use CMD-. (in Jupyter Command mode).
* It is a shortcut for

In [ ]:
sc.cmd('s.freeAll')

which is also available via

In [ ]:
sc.free_all()

Use raw python strings for multiline sc3-programs:

In [ ]:
sc.cmd(r"""
Routine({
    x = 10.collect{ |i|
        0.2.wait;
        Synth.new(\default, [\freq, 50+(50*i)]);
    };
    2.wait;
    x.do{|e| 
        e.release;
        0.1.wait;};
}).play;
""")

Python variables can be injected into sc3 commands by using the ^ special: The following examples demonstrates it by setting frequencies by using python variables

In [ ]:
for p in range(1, 50):  # a tone ladder
    freq = 50 + p*3
    dur = np.log(p)
    position = np.sign(p-25)
    %sc Synth.new(\s1, [\freq, ^freq, \dur, ^dur, \pan, ^position])
    time.sleep(0.05)

This works only for simple variable types (TODO: describe, test, extend).

The same can be achieved by using the cmd|cmdv|cmdg() functions by providing a dictionary of variable names and content (which can use other python vars or statements)

In [ ]:
a = 15
sc.cmdv("^name1 + ^name2", pyvars={'name1': a+9,'name2': 18})

In [ ]:
# without pyvars, global variable are used.
freq = 5
rate = 6
sc.cmdv("(^freq + 1) * (^rate + 1)")

**Getting SC output to python**

* To get the output of an sc program into a python variable, use the cmdg function.
* The following example shows how to transfer a synth's nodeID

In [ ]:
sc.cmd(r"""x = Synth.new(\default)""")

In [ ]:
nodeID = sc.cmdg("x.nodeID")  # get the nodeId to python
print(nodeID)

In [ ]:
sc.msg("/n_free", nodeID)  # here the nodeID is used to free the instance via scsynth

**sc.cmdg(), resp. %scg now return integers, floats, strings and lists**
* %scg can be assigned to a python variable within code
* BUG: %scg command must not include quotes '"', 
    * they conflict with the r function, where the string is interpreted
    * bugfix is to send sc prog and assign return value to a variable, e.g. ~ret
    * then use %scg simply to get ~ret (see example below)

In [ ]:
a = %scg 1234+23452
print(f"returned an {type(a)} of value {a}")

In [ ]:
%scg 1234.5.squared

In [ ]:
%scg "sonification".scramble

In [ ]:
%sc ~retval = "sonification".scramble
#print(sc.osc.returns)
%scg ~retval

In [ ]:
a = %scg (1,1.1..2)
print(f"list with length: {len(a)}")
a

Note that floating-point numbers do have a limited precision

In [ ]:
[round(num, 6) for num in a]

## Cell magics

In [ ]:
%sc {SinOsc.ar(MouseX.kr(200,400))}.play  // move mouse horizontally, CMD-. to stop

In [ ]:
%sc s.scope()

In [ ]:
value = %scg 75-25
print("value = ", value)

**BUG: old stuff sits and is returned as value - howto discard???**

In [ ]:
%%sc
{
    x = Synth.new(\s2, [\freq, 100, \num, 1]);
    500.do{|i|
        x.set(\freq, sin(0.2*i.pow(1.5))*100 + 200);
        0.02.wait;
    };
    x.free;
}.fork

* Try %scv and %%scv for verbose line resp. cells
* Try %scg and %%scg for getter line resp. cells

## exit - startup - boot - delete

In [ ]:
sc.exit()  # shuts down the server and closes the sclang subprocess

In [ ]:
sc = scn.startup()

* The server can be booted with sc.boot().
* This is normally not necessary as it happens automatically with startup()
* However, if the server would be down, e.g. after a s.quit, it can be booted in this way. Compared to an sc.cmd("s.boot"), the sc.boot() waits until booting as completed.

In [ ]:
%sc s.quit  // let us quit the server first

In [ ]:
sc.boot_with_blip()

boots the server. This is not needed generally as it happens with startup() automatically

sc.boot_with_blip() is a special boot function that executes some custom code when booted. It is by default executed on scn.startup(). 

It creates
* a synthdef "s1" is created which is a discrete sound event with parameters 
    * frequency freq
    * duration dur
    * attack time att
    * amplitude amp
    * number of harmonics num
    * spatial panning pan
* synthdef "s2" is created which is a continuous synth with parameters
    * frequency freq
    * amplitude amp
    * number of harmonics num
    * spatial panning pan
* a synthdef "record" is created with parameter bufnum (buffer number), which simply records audio input to a buffer
* finally, two test tones, one with "s1", one with "s2" are created, which should relax any tensions whether sc is up and running.


In [ ]:
# del(sc) --> shouldn't do it, or?
# __s_quit() --> just a private function

## scsynth control

* Communication with scsynth is done via OSC using the cross-platform package python-osc.
* Direct control of synths shortcuts a detour via sclang and is both more efficient and promises a lower latency, however, at the cost of less convenience.
* The basic wrapper for OSC messages, both to scsynth and sclang is 

* By default, it sends messages to scsynth, but with sclang=True messages can be sent to sclangs OSC port.

In [ ]:
sc.msg("/s_new", ["s1", 1001, 1, 0, "freq", 300])  # a short 300 Hz tone

* note that you have to specify the nodeID (here 1001), possibly without knowing whether that nodeID is free. If it is not, the node could not be created.
* using nodeID -1 will let scsynth automatically select a free nodeID, however, so far there is no communication channel to receive this number from scsynth

In [ ]:
# a more complex example
for p in [0,2,4,7,5,5,9,7,7,12,11,12,7,4,0,2,4,5,7,9,7,5,4,2,4,0,-1,0,2,-5,-1,2,5,4,2,4]:
    freq = scn.midicps(60+p)  # see helper fns below
    sc.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "dur", 0.5, "num", 1])
    time.sleep(0.15)

* Note that the timing is here under python's control, which is not very precise.
* Bundles allow to specify a timetag and thus let scsynth control the timing, which is much better, if applicable. Bundles can be sent by

In [ ]:
sc.bundle(2.2, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1])  # a tone starts in 2.2s
sc.bundle(2.7, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1])  # a tone starts in 2.7s

* small numbers are times in seconds relative to time.time() evaluated at fn execution
* use time.time()+timeoffset to specify in absolute times (see next example)

In [ ]:
t0 = time.time()
for i, r in enumerate(np.random.randn(200)):
    onset = t0 + 2 + r
    freq = 500 + 5 * i
    sc.bundle(onset, "/s_new", ["s1", -1, 1, 0, 
                                "freq", freq, "dur", 1.5, "num", abs(r)+1]) 

**Remarks**:
* note that the python code returns immediately and all events remain in scsynth
* note that unfortunately scsynth has a limited buffer for OSC messages, so it is not viable to spawn thousends of events. scsynth will then simply reject OSC messages.
* this motivated (and is solved) with a TimedQueue, see below.

## Recording sc3 output into file with scsynth

The following three functions provide a simple interface using direct msgs to scsynth to record any sc3 output into an audio file

In [ ]:
# open a oscilloscope and synth that plays back lp filtered microphone signal
%sc {LPF.ar(SoundIn.ar(0), 1500, 1)}.scope  

In [ ]:
sc.prepare_for_record(0, "my_recording.wav", 99, 2, "wav", "int16")  # buffer 99 will be used
t0 = time.time()
sc.record(t0+0.1, 2001)  # recording starts in 200 ms
sc.bundle(0.2, "/s_new", ["s1", -1, 1, 1, "freq", 200, "dur", 1])  
sc.bundle(0.5, "/s_new", ["s1", -1, 1, 1, "freq", 300, "dur", 1])
sc.stop_recording(t0+2.0) # and stops in 1 seconds

* note that the sorting in scsynth node tree is with 'at begin' rule
* otherwise the rendered tones would be rendered after the outbus was written to file
* resulting in an empty file.
* however, any whistling recorded via the microphone should be in the file.
* the file appear in the same folder as this .ipynb file.

## MIDI interface

In [ ]:
midi_ctrl_synth(self, synthname='\\syn')

In [ ]:
midi_ctrl_free(self)

In [ ]:
midi_gate_synth(self, synthname='\\syn')

In [ ]:
midi_gate_free(self)

## Helper functions

* SuperCollider coders are familiar and frequently use a number of useful converter functions
* the helper functions provide pythonic pendants namely currently for (to be extended):

* to linearly map x from between [smi, sma] to [dmi, dma]
* non range check is done, and no clipping.
* for negative slope mapping, simply swap dmi and dma

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
xs = np.linspace(1,9,100)
plt.plot(xs, [scn.linlin(x, 0, 10, 300, 500) for x in xs])
plt.axis([0,10,300,500]);

In [ ]:
scn.midicps(69.2)  # convert MIDI note to cycles per second (cps) in [Hz]

In [ ]:
scn.cpsmidi(440)   # and back to MIDI note (in float resolution)

In [ ]:
xs = np.linspace(1,9,100)
plt.plot([scn.clip(x, 5, 7) for x in xs]);

In [ ]:
# dbamp(db) converts dB value in amplitude, 0 dB = 1, '*2' \approx +6dB
dbs = np.linspace(-20, 20)
plt.plot(dbs, [scn.dbamp(d) for d in dbs]);
# plt.semilogy()

In [ ]:
# ampdb(amp) converts an amplitude to dB, assuming 0dB=1
scn.ampdb(0.2)

## Creating an OSC responder and msg to sclang for synthesis

In [ ]:
%%scv
OSCdef(\dinger, { | msg, time, addr, recvPort |
    var freq = msg[2];
    // msg.postln;
    {Pulse.ar(freq, 0.04, 0.3)!2 * EnvGen.ar(Env.perc, doneAction:2)}.play
}, '/ding')

In [ ]:
for i in range(100):
    sc.msg("/ding", ["freq", 1000-5*i], sclang=True)

In [ ]:
%sc OSCdef.freeAll()

## TimedQueue Basics

Motivation:
* for sonifications with precise timing, thousends of events need to be spawn at the exact time.
* doing this with bundles doesn't work as the OSC buffer of scsynth is limited
* it needs a TimedQueue where events can be added for time-precise dispatching
* a thread then simply checks what items are due and executes them
* using arbitrary functions as objects for the queue allows to use it both for sonification (e.g. sending OSC messages/bundles) and even visualization
* however, the functions should complete really quickly as otherwise the queue would run late and fail to process due events
* hence, it is the responsibility of the user to be careful
* if, however, longer programs are needed, they can be spawn as threads on execution
* The following demos illustrate the core functionality with console print and sound

In [ ]:
queue = scn.TimedQueue()

In [ ]:
import sys
def myfun(x):
    os.write(1, "{}\n".format(x).encode())
    sys.stderr.flush()
    
def myblip(freq):
    sc.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "num", 3])

In [ ]:
myfun(4)

In [ ]:
myblip(700)

In [ ]:
t0 = time.time()
for i in range(50):
    queue.put(t0+i*0.04, myblip, 800+1*7*i)
    queue.put(t0+i*0.04, myfun,  400+30*i)  # plots on stderr = console
print(time.time()-t0)

In [ ]:
queue.close()

## TimedQueueSC example with synchronized sound and mpl plot

* This example shows how to highlight data points as they are played.
* However, the marking is reset for every new data point, i.e. data points are not highlighted as long as the corresponding sound lasts
* to achieve that, see code example below

**Note that there are sometimes some strange effects with matplotlib event loop hickups in Mac, it is untested with Linux or Windows, any problem reports or suggested solutions are welcome.**

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt5

In [ ]:
%matplotlib  

In [ ]:
# load data
data = np.vstack((np.random.randn(50, 5), np.random.randn(100, 5)+3.5))

In [ ]:
# create figure, axis, plots
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); mngr.window.setGeometry(1200, 0, 500, 400)
pldata, = ax.plot(data[:,1], data[:,2], ".", ms=5) # create plots
plmarked, = ax.plot([], [], "ro", ms=5, lw=0.5)
# plt.show(block=False); plt.ion(); fig.canvas.draw() # not needed if plot shows

# create the queue
queue = scn.TimedQueueSC(sc)

In [ ]:
def update_plot(x, y):
    global fig, ax, pldata, plmarked
    plmarked.set_data([x], [y])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(plmarked)
    fig.canvas.update() # additional fig.canvas.flush_events() not needed?

t0 = time.time()
for i, r in enumerate(data):
    onset = t0 + scn.linlin(r[1], 4, 8, 0.5, 4) + random.random()*0.2
    freq = scn.midicps(scn.linlin(r[2], 2, 5, 60, 80))
    pos = scn.linlin(r[4], 0, 2, -1, 1)
    queue.put(onset-0.1, sc.bundle, (onset, "/s_new", [
        "s1", -1, 1, 0, "freq", freq, "amp", 0.05, "dur", .52, "pos", pos]))
    queue.put(onset-0.1, update_plot, (r[1], r[2]), spawn=False)
print('time used:', time.time() - t0)

## TimedQueueSC PMSon with matplotlib highlights

* this example illustrates how to use TimedQueues to maintain a 'currently playing selection' of data points, so that the GUI highlight gets deactivated as the corresponding sound stops
* this is achieved by scheduling a select and unselect function invocation at the time stamps where the corresponding sound starts and ends
* Note that herre the actual plot update is in a second loop of scheduled 'update_plot' invocations at an independent user controlled frame rate

In [ ]:
data = np.vstack((np.random.randn(300, 7), np.random.randn(300, 7)+5))

In [ ]:
# create figure
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); mngr.window.setGeometry(1200, 0, 500, 400)

# create the queue
queue = scn.TimedQueueSC(sc)

In [ ]:
def mapcol(row, stats, col, val_from, val_to):  # helper for mapping
    return scn.linlin(row[col], stats[col, 0], stats[col, 1], val_from, val_to)

def select(i):  #  highlight selection
    selected[i] = True

def unselect(i):
    selected[i] = False

def update_plot(xs, ys): 
    global fig, ax, pldata, plmarked, selected
    plmarked.set_data(xs[selected], ys[selected])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(plmarked)
    fig.canvas.flush_events()
    fig.canvas.update()

# parameter mapping sonification with GUI
tot_dur = 5  # total duration of the sonification
max_ev_dur = 5.5  # maximal event duration
delay = 1  # offset

stats = np.vstack((np.min(data, 0), np.max(data, 0))).T
selected = np.zeros(np.shape(data)[0], np.bool)

# create axis, plots
ax.clear()
plmarked, = ax.plot([], [], "ro", ms=4, lw=0.5)
pldata, = ax.plot(data[:,1], data[:,2], ".", ms=2) # create plots

t0 = time.time()

for i, r in enumerate(data):
    onset = t0 + delay + 5* i/800 # mapcol(r, stats, 3, 0, tot_dur)
    freq  = scn.midicps( mapcol(r, stats, 2, 60, 90))
    ev_dur = mapcol(r, stats, 4, 0.2, max_ev_dur)
    # sonification
    queue.put(onset-delay, sc.bundle, (onset, "/s_new", [
        "s1", -1, 1, 0, "freq", freq, "amp", 0.05, "dur", ev_dur, "pos", pos]))
    # on/off events of marker highlight
    queue.put(onset, select, i)
    queue.put(onset + ev_dur, unselect, i)

# update plot at given rate from earliest to latext time
for t in np.arange(t0, t0+delay+tot_dur+ev_dur+1, 1/10):  # 1 / update rate
    queue.put(t, update_plot, (data[:,1], data[:,2]))

## TimedQueueSC PMSon with timeseries data and matplotlib

* The following example illustrates howto create a continuous sonification with concurrent plotting the time in a plot
* This presumes time-indexable data
* a maximum onset variable is maintained to shutdown the continuously playing synths when done
* note that the highlight will only replot the marker, required time is thus independent of the amount of data plotted in the other plot.

In [ ]:
sc.bundle(0, "/s_new", ["s2", 1200, 1, 0, "amp", 0.04])
sc.bundle(0.2, "/n_set", [1200, "freq", 200, "num", 1, "amp", 0.2, "pan", 0])
sc.bundle(0.4, "/n_free", 1200)

In [ ]:
ts = np.arange(0, 20, 0.01)
data = np.vstack((ts, 
                  np.sin(2.5*ts) + 0.01*ts*np.random.randn(np.shape(ts)[0]), 
                  0.08*ts[::-1]*np.cos(3.5*ts)**2)).T

In [ ]:
# create figure
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); mngr.window.setGeometry(1200, 0, 500, 400)

# create axis, plots
ax.clear()
plmarked, = ax.plot([], [], "r-", lw=1)
pldata, = ax.plot(data[:,0], data[:,1], "-", ms=2) # create plots
pldataR, = ax.plot(data[:,0], data[:,2], "-", ms=2) # create plots

In [ ]:
# create the queue
queue = scn.TimedQueueSC(sc)

def mapcol(row, stats, col, val_from, val_to):  # helper for mapping
    return scn.linlin(row[col], stats[col, 0], stats[col, 1], val_from, val_to)

def update_plot(t): 
    global fig, ax, pldata, plmarked, selected
    plmarked.set_data([t,t], [-10000, 10000])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(pldataR)
    ax.draw_artist(plmarked)
    fig.canvas.update()
    # fig.canvas.flush_events()

stats = np.vstack((np.min(data, 0), np.max(data, 0))).T
selected = np.zeros(np.shape(data)[0], np.bool)

# parameter mapping sonification with GUI
delay = 0.5
rate = 2

t0 = time.time()
queue.put(t0, sc.msg, ("/s_new", ["s2", 1200, 1, 0, "amp", 0]))
queue.put(t0, sc.msg, ("/s_new", ["s2", 1201, 1, 0, "amp", 0]))

max_onset = 0
latest_gui_onset = 0
gui_frame_rate = 20

ts = []
for i, r in enumerate(data[::2, :]):
    ts.append(time.time()-t0)
    if i==0: tmin = r[0]
    onset = (r[0]-tmin)/rate
    freq   = scn.midicps( mapcol(r, stats, 1, 60, 70))
    freqR  = 0.5 * scn.midicps( mapcol(r, stats, 2, 70, 80))

    # sonification
    tt = t0 + delay + onset
    if tt > max_onset: max_onset = tt
    queue.put(tt-0.1, sc.bundle, (tt, "/n_set", 
        [1200, "freq", freq, "num", 4, "amp", 0.2, "pan", -1, "lg", 0]))
    queue.put(tt-0.1, sc.bundle, (tt, "/n_set", 
        [1201, "freq", freqR, "num", 1, "amp", 0.1, "pan", 1]))
    if tt > latest_gui_onset + 1/gui_frame_rate:  # not more than needed gui updates
        latest_gui_onset = tt
        queue.put(tt-0.1, update_plot, (r[0],), spawn=False)
queue.put(max_onset, sc.msg, ("/n_free", 1200))
queue.put(max_onset, sc.msg, ("/n_free", 1201))
              
# queue.join()
print(time.time()-t0)